In [1]:
from keras.models import load_model
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import matplotlib.pyplot as plt

new_df = pd.read_csv('new_gold.csv')

# Assuming new_df is your DataFrame and it's already preprocessed
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df['Date'] = new_df['Date'].apply(lambda x: x.toordinal())

new_df.set_index('Date', inplace=True)

# Normalize the Dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(new_df)

# Create the input data and the target variable
X, y = [], []
for i in range(len(scaled_data)-1):
    X.append(scaled_data[i])
    y.append(scaled_data[i + 1])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Create the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compile the LSTM network
model.compile(loss='mean_squared_error', optimizer='adam')

# Create callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Fit the LSTM network with validation split
model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=2, callbacks=[es, mc], validation_split=0.2)

model.save('gold_price.h5')

# Load the model
model = load_model('gold_price.h5')

# Get user input for the year
year_input = input("Enter the year (YYYY): ")

# Load the training set
training_set = pd.read_csv('new_gold.csv')

# Preprocess the training set
training_set['Date'] = pd.to_datetime(training_set['Date'])
training_set['Date'] = training_set['Date'].apply(lambda x: x.toordinal())
training_set.set_index('Date', inplace=True)
scaled_training_set = scaler.transform(training_set)

# Prepare the input data for prediction
X_train_pred = []
for i in range(len(scaled_training_set)-1):
    X_train_pred.append(scaled_training_set[i])

X_train_pred = np.array(X_train_pred)

# Predict the gold price for the given year
predicted_price = model.predict(np.reshape(X_train_pred[-1], (1, X_train_pred.shape[1], 1)))

# Inverse transform the predicted price
predicted_price = scaler.inverse_transform(predicted_price)

# Print the predicted price
print("Predicted gold price for year", year_input, ":", predicted_price[0][0])

C:\Users\Lynn\AppData\Local\Temp\ipykernel_2520\1438411855.py:16: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_df['Date'] = pd.to_datetime(new_df['Date'])




Epoch 1/1000


Epoch 1: val_loss improved from inf to 0.42090, saving model to best_model.h5
8/8 - 3s - loss: 0.0236 - val_loss: 0.4209 - 3s/epoch - 407ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 0.42090 to 0.35950, saving model to best_model.h5
8/8 - 0s - loss: 0.0176 - val_loss: 0.3595 - 63ms/epoch - 8ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 0.35950 to 0.29931, saving model to best_model.h5
8/8 - 0s - loss: 0.0135 - val_loss: 0.2993 - 64ms/epoch - 8ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 0.29931 to 0.25164, saving model to best_model.h5


c:\Users\Lynn\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 - 0s - loss: 0.0117 - val_loss: 0.2516 - 65ms/epoch - 8ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 0.25164 to 0.22153, saving model to best_model.h5
8/8 - 0s - loss: 0.0115 - val_loss: 0.2215 - 65ms/epoch - 8ms/step
Epoch 6/1000

Epoch 6: val_loss improved from 0.22153 to 0.21460, saving model to best_model.h5
8/8 - 0s - loss: 0.0106 - val_loss: 0.2146 - 66ms/epoch - 8ms/step
Epoch 7/1000

Epoch 7: val_loss improved from 0.21460 to 0.20764, saving model to best_model.h5
8/8 - 0s - loss: 0.0095 - val_loss: 0.2076 - 68ms/epoch - 9ms/step
Epoch 8/1000

Epoch 8: val_loss improved from 0.20764 to 0.17723, saving model to best_model.h5
8/8 - 0s - loss: 0.0085 - val_loss: 0.1772 - 74ms/epoch - 9ms/step
Epoch 9/1000

Epoch 9: val_loss improved from 0.17723 to 0.13151, saving model to best_model.h5
8/8 - 0s - loss: 0.0072 - val_loss: 0.1315 - 68ms/epoch - 9ms/step
Epoch 10/1000

Epoch 10: val_loss improved from 0.13151 to 0.09256, saving model to best_model.h5
8/8 - 0s - loss: 0.0

C:\Users\Lynn\AppData\Local\Temp\ipykernel_2520\1438411855.py:64: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  training_set['Date'] = pd.to_datetime(training_set['Date'])


1/1 [==============================] - 1s 561ms/step
Predicted gold price for year 2500 : 1871.9025
